# Testing the audio

**Note:** 
- It is advisable to read the README file before running this notebook.
- Also stop all other running Python script or notebook connected to the robot as only one connection can run at the same time.

# Imports

First we start by importing the necessary modules for this notebook :

In [ ]:
import time
import cv2 as cv
import matplotlib.pyplot as plt
import reachyAudio
from reachy import Reachy, parts

## Instantiate a ReachyAudio object

In [ ]:
reachy_audio = reachyAudio.ReachyAudio()

## Test : Recording

To record audio samples and save them as a WAV file, you can use the method named recordAudio :

In [ ]:
reachy_audio.recordAudio()

By default, the record time is five seconds and the name of the output file is "output.wav".
However, it is possible to specify them :

In [ ]:
reachy_audio.recordAudio(recordTime= 8, wavOutputFileName= "test.wav")

## Test : Play a wave file

To play a WAV file, you can use the method named playAudio :

In [ ]:
reachy_audio.playAudio("output.wav")

## Test : text to speech

To make Reachy talk, the method speak can be used :

In [ ]:
reachy_audio.speak("Hello there, my name is Reachy")

It is possible to specify the rate (default is 150) and the volume (default is 1.0) of the voice :

In [ ]:
reachy_audio.setEngineProperties(rate= 200, volume= 0.8)
reachy_audio.speak("Hello there, my name is Reachy")

Finally, one can also specify the desired voice (the default one is english). First we check what are the available voices :

In [ ]:
reachy_audio.availableVoices()

Then, we can test one of these voices :

In [ ]:
reachy_audio.setEngineProperties(voice_id= "italian")
reachy_audio.speak("Buongiorno, sono il robot Reachy")
reachy_audio.setEngineProperties()

## Test : Speech recognition

To recognize a speech, one can use the method named recognizeSpeech.
The recognizer will wait until the user says something and will stop the listening when the user stops to speak. 

In [ ]:
said = reachy_audio.recognizeSpeech()

Here is a simple chatting interaction :

In [ ]:
if "hello" in said:
    reachy_audio.speak("Hello there, my name is reachy.")

## Test : voice activity detection

To detect voice activity, you can simply call the method named is_voice. This method will return a boolean indicating if someone is speaking or not.

In [ ]:
print(reachy_audio.mic.is_voice())  

It is also possible to call the method longIsVoice. This method allow you to store in a list the result of several calls to isVoice spaced by a delay.

In [ ]:
recordingVoiceActivity = reachy_audio.longIsVoice(numberMeasures= 30, timeDelay= 0.2)
print(recordingVoiceActivity)

## Test : sound localization

To detect sound orientation, you can simply call the method named direction. This method will return an angle in degrees representing the direction of arrival of the incomming sound.
The angle provided follow a counter clockwise orientation.

In [ ]:
print(reachy_audio.mic.direction)

You can also use the longSoundOrientation method that will behave similarly to the longIsVoice method.

In [ ]:
recordingSoundOrientation = reachy_audio.longSoundOrientation(numberMeasures= 40, timeDelay= 0.05)
print(recordingSoundOrientation)

### Orient the head toward interlocutor

First we need to connect to the head :

In [ ]:
reachy = Reachy(head=parts.Head(io='/dev/ttyUSB*'),)

Check that all 3 disks are present and ok.

In [ ]:
for d in reachy.head.neck.disks:
    print(d, d.temperature)

Start by going to base position :

In [ ]:
reachy.head.compliant = False
reachy.head.look_at(1, 0, 0, duration=2, wait=True)

Then we can start to make Reachy's head follow the direction of the interlocutor's voice :

In [ ]:
reachy_audio.orientToInterlocutor(reachy)

success, img = reachy.head.right_camera.read()
if success:
    plt.figure()
    plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

In [ ]:
reachy.head.compliant = True

## Test : LEDs control

It is possible to control the LEDs of the microphone array of Reachy. The use of theses leds can for example significantly improve the human computer interaction by giving feedback to the interlocutor such as the internal state of the robot (listening, processing data, answering, etc...). It can also be used for any other purposes. Here is an overview of the availables functions :

To set all the LEDs to a single color, one can use the method named mono :

In [ ]:
reachy_audio.pixel_ring.mono(0xFF0000)
time.sleep(3)
reachy_audio.pixel_ring.mono(0x00FF00)
time.sleep(3)
reachy_audio.pixel_ring.mono(0x0000FF)
time.sleep(3)
reachy_audio.pixel_ring.mono(0xFF00FF)

To avoid the use hexadecimal values, one can also use the set_color method as follows :

In [ ]:
reachy_audio.pixel_ring.set_color(r=255, g=0, b=0)

The trace method allows the LEDs to change depending on the voice activity detection (VAD) and the direction of arrival angle (DOA) : 

In [ ]:
reachy_audio.pixel_ring.trace()

The listen method is similar to the trace method except that the LEDs don't turn off :

In [ ]:
reachy_audio.pixel_ring.listen()

Others additional features are the think method, the speak method and the spin method :

In [ ]:
reachy_audio.pixel_ring.think()

In [ ]:
reachy_audio.pixel_ring.speak()

In [ ]:
reachy_audio.pixel_ring.spin()

It is possible to set the color palette :

In [ ]:
reachy_audio.pixel_ring.set_color_palette(0x00FF00, 0xFFFF00)
reachy_audio.pixel_ring.speak()
time.sleep(3)
reachy_audio.pixel_ring.set_color_palette(0xFF0000, 0xFF00FF)
reachy_audio.pixel_ring.speak()

One can also set the brightness of the LEDs (range: 0x00 - 0x1F) :

In [ ]:
reachy_audio.pixel_ring.set_brightness(0x12)
reachy_audio.pixel_ring.speak()
time.sleep(3)
reachy_audio.pixel_ring.set_brightness(0x05)
reachy_audio.pixel_ring.speak()

One can set how the VAD LED should operate (0 = off, 1 = on, else = depends on VAD):

In [ ]:
reachy_audio.pixel_ring.set_vad_led(0)
time.sleep(3)
reachy_audio.pixel_ring.set_vad_led(1)
time.sleep(3)
reachy_audio.pixel_ring.set_vad_led(2)

An additional feature is the set_volume method (range: 0 - 11, no relation with the volume of the speaker):

In [ ]:
reachy_audio.pixel_ring.set_volume(0)
time.sleep(3)
reachy_audio.pixel_ring.set_volume(6)
time.sleep(3)
reachy_audio.pixel_ring.set_volume(11)

Finnaly, one can turn off the LEDs by using the method named off :

In [ ]:
reachy_audio.pixel_ring.off()

## Test : Conversation

To allow Reachy to do a conversation with people, one can call the method named conversation.

**Note:** 
- The fan used to cool the orbita system motors is quite close to the microphone array and thus induces some noise. Better results for the conversation can be obtained by doing the calibration of the recognizer while this fan is on, so that the recognizer considers this noise as an ambient noise for which he is calibrated. The calibration step is automatically called during the initialization of the reachyAudio object, however one can also manually ask for a new calibration of the recognizer by calling the method named calibrateRecognizer.
- To stop the conversation method, use the interrupt kernel button (button located on the right side of the run button). If you are using the conversation method in a .py file and not in a jupyter notebook, you can stop the method simply by pressing a key on the keyboard.

In [ ]:
# Make sure to run this cell when the orbita system fan is on.
reachy_audio.calibrateRecognizer()

In [ ]:
reachy.head.compliant = False
reachy.head.look_at(1, 0, 0, duration=2, wait=True)

In [ ]:
reachy_audio.conversation(reachy)

In [ ]:
reachy.head.compliant = True